In [1]:
import pandas as pd
import numpy as np

In [14]:
# load data
train = pd.read_csv('../data/merged_data/train.csv', dtype={'INSTANCE_ID': object,
                                                      'INCIDENT_NUMBER': object,
                                                      'INCIDENT_TYPE_ID': object,
                                                      'MILLESIME': object,
                                                      'RESOURCE_ID':object,
                                                      'LOCATION_ID': object,
                                                      'ORGANISATION_ID': object,
                                                      'PARTY_ID_OCC': object,
                                                      'MARQUE_CODE': object,
                                                      'CODE_POSTAL': object,
                                                      'L1_ORGANISATION_ID': object,
                                                      'CIA': object,
                                                      'ORGANISATION_CODE': object
                                                     })

/Users/WM1158/anaconda2/envs/py3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
train = pd.read_pickle('../data/merged_data/train.pkl')
test = pd.read_pickle('../data/merged_data/test.pkl')

In [3]:
y_train = train['target']
#y_test = test['target']

In [4]:
drop = ['INSTANCE_ID', 'INCIDENT_NUMBER']
drop_atm = ['AUTEUR_INCIDENT',
            'MARQUE_LIB',
            'OPTION',
            'MODELE_CODE',
            'COMMENTAIRE_BI', 
            'RESOURCE_ID', 
            'SCHEDULED_START_DATE', 
            'SCHEDULED_END_DATE', 
            'CRE_DATE_GZL',
            'INSTALL_DATE',
            'CODE_POSTAL',
            'L2_ORGA_CODE_POSTAL',
            'UPD_DATE',
            'DATE_DEBUT',
            'DATE_FIN' ]

In [5]:
train.drop(drop + drop_atm + ['target'], axis=1, inplace=True)
test.drop(drop + drop_atm, axis=1, inplace=True)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048030 entries, 0 to 1048029
Data columns (total 38 columns):
INCIDENT_TYPE_ID       1048030 non-null object
INCIDENT_TYPE_NAME     1048030 non-null object
TYPE_BI                1048030 non-null object
NB_PASSAGE             1048030 non-null float64
MILLESIME              1048030 non-null object
PROBLEM_CODE           1048030 non-null object
ORIGINE_INCIDENT       1048030 non-null object
TYPE_UT                1048030 non-null object
GRAVITE                1048030 non-null object
ORGANISATION_ID        1048030 non-null object
TYPE_OCC               1048030 non-null object
RACHAT_CODE            1048030 non-null object
NATURE_CODE            1048030 non-null object
USAGE_LOCAL            1048023 non-null object
PAYS                   1048030 non-null object
TYPE_VOIE              1048030 non-null object
POINTS_FIDEL           1048030 non-null float64
STOP_PHONING           1048030 non-null object
CODE_GEN_EQUIPEMENT    1048030 non-null

In [7]:
categoricals = list(train.columns[train.dtypes == object])

In [18]:
# preprocess
train.dropna(inplace=True)
# change dates in timedeltas



In [19]:
# label encode the categorical features of the train dataset
from collections import defaultdict
d = defaultdict(LabelEncoder)
categoricals_train = train[categoricals]
categoricals_test = test[categoricals]

In [20]:
#Encoding the variable
fit = categoricals_train.apply(lambda x: d[x.name].fit_transform(x))

In [ ]:
# Inverse the encoded
fit.apply(lambda x: d[x.name].inverse_transform(x))

In [22]:
# Using the dictionary to label future data
categoricals_test.apply(lambda x: d[x.name].transform(x))

ValueError: ("y contains new labels: ['Fax']", 'occurred at index ORIGINE_INCIDENT')

In [9]:
from sklearn.feature_extraction import DictVectorizer
 
def encode_onehot(df, cols):
    """
    One-hot encoding is applied to columns specified in a pandas DataFrame.
    
    Modified from: https://gist.github.com/kljensen/5452382
    
    Details:
    
    http://en.wikipedia.org/wiki/One-hot
    http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(orient='records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df

In [12]:
onehot = encode_onehot(train, categoricals)

In [13]:
onehot

,NB_PASSAGE,POINTS_FIDEL,CONTRAT_TARIF,PRIX_FACTURE,CIA,CIA=2112,CIA=2114,CIA=2118,CIA=2120,CIA=2121,...,TYPE_VOIE=TOUR,TYPE_VOIE=TRA,TYPE_VOIE=VAL,TYPE_VOIE=VEN,TYPE_VOIE=VIA,TYPE_VOIE=VLA,TYPE_VOIE=VOI,USAGE_LOCAL,USAGE_LOCAL=HABI,USAGE_LOCAL=PRO
0,1.0,0.0,142,71,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,450.0,251,251,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2.0,350.0,135,102,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,900.0,262,305,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,300.0,247,61,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,1.0,50.0,163,163,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,1.0,100.0,145,118,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,1.0,350.0,243,149,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,1.0,500.0,147,123,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,1.0,850.0,251,216,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
#One hot encoding of categorical variables
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


In [47]:
# label encoder (one variable at a time)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(train[categoricals])

ValueError: bad input shape (1048030, 37)

In [ ]:
# binary encode
onehot_encoder = OneHotEncoder()
#integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

In [45]:

enc = OneHotEncoder()

oneHot = enc.fit_transform()

ValueError: could not convert string to float: 'TIRAGE NATUREL'

In [ ]:
# feature ingineering

# age installation
# temps depuis dernière visite (pas forcément dispo sur le test)
# déjà eu une casse sur ce matériel
# temps depuis dernière casse
# convertir toutes les dates en timedelta
# extraire des éléments des dates (année, mois, jour, heure, jour de la semaine intervention, heure ?)


In [ ]:
# model

In [ ]:
# prediction on test set